<a href="https://colab.research.google.com/github/Gaianeve/The_real_Fufi/blob/main/simple_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's try moving our Fufi 🐶 with Arduino 💻


In [ ]:
!pip install serial

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 759.1 kB/s eta 0:00:00


In [ ]:
#importing libraries
import numpy
import serial
import time
import struct

## Setting up communication with Arduino ✨ 🎙

In [ ]:
#set up connection with Arduino
arduino = serial.Serial(
    port='/dev/ttyUSB1',
    baudrate=9600,
    timeout = 1
)


 **Detailed explanation of the code** 🤯
```
arduino = serial.Serial(
    port=/dev/ttyUSB1,
    baudrate=9600,
    timeout = 1
)

```
*  ``` port=/dev/ttyUSB1```: This specifies the serial port to which you are connecting. ```/dev/ttyS1``` is a common serial port identifier in Unix-like operating systems (such as Linux).
*  ```baudrate=9600```: This sets the baud rate for the connection to 19200 bits per second
* ``` timeout = 1 ```: This sets a read timeout of 1 second. If no data is received within this time frame, the read operation will return


 **How can I know the exact port Arduino is connected to?** 🤯
* Python script (all operative systems)
```
  import serial.tools.list_ports

  ports = serial.tools.list_ports.comports()
  # list all available serial ports
  for port in ports:
      print(port.device)
```

* Linux terminal
  1.   Command line ```ls /dev/tty*```;

    Look for a device named something like ```/dev/ttyUSB0``` or ```/dev/ttyACM0```. The exact name will vary depending on the model of your Arduino and the USB port it's connected to.
  2.   Command line ```dmesg | grep tty```;

    Run the dmesg command right after plugging in your Arduino. It will show the kernel messages, including information about the newly connected device. What's after dmseg is only needed to filter the output to see relevant information.




## Collect data 🙏 🛕

In [ ]:
def get_measure(old_theta):
  start_time = time.time() #measure starts
  old_theta = 0 #previous angle

  # read Arduino measurements (not sure about the form)
  arduino_red = arduino.read(12) # read 12 byte
  acc, gyro = struct.unpack("<iiL", arduino_red) #give me the measure in the format I want
  end_time = time.time() #measure ends
  dt = end_time - start_time

  #compute pole angle
  theta = 0.98*(old_theta + (gyro[0]/131)*dt) + 0.02*acc[0] #complementary filter: 98% gyro and 2% acc
  return acc, gyro, theta, dt

 **Explanation** 🤯

The `struct.unpack ` function in Python's struct module is used to unpack data from a binary string (or bytes object) according to a specified format. Let's break down the usage and format string you provided: `<iiL`.

1. `<`: Indicates that the data is in little-endian byte order.
2. `i`: Represents a signed integer (4 bytes).
3. `i`: Another signed integer (4 bytes).
4. `L`: Represents an unsigned long integer (4 bytes on many systems, but it can be 8 bytes on some systems).

So, the format string `<iiL` tells `struct.unpack` to interpret the binary data in `arduino_red` as consisting of:

*   Two 4-byte signed integers;
*   One unsigned long integer (4 bytes, if we're assuming typical 32-bit size);

🤔  In our specific case, we should change it according to what we get from arduino.

Here I assume Arduino read gives six numer, acceleration for the accelerometer and angular velocity from gyro. We should do a practical example of that to fix it. I guess it reads directly all of the output pins, but not sure about it (or the order).

Also not sure about when to start and end taking time for the data acquisition 🤔

## Give instruction to FUFI and get reaction 🪖 🐶


In [ ]:
def step(motor_command, old_theta):
  direction =  motor_command > 0
  # convert motor command to 16 bit unsigned int
  int_motor_command = int(np.abs(motor_command) * (2**16 - 1))

  #registers thing: write the action to do on the motor pin
  tx = b"\x10\x02"  # start sequence
  tx += b"\x01"  # command type = STEP = 0x01
  tx += struct.pack("<?H", direction, int_motor_command) #pack our command in a form that arduino can get
  arduino.write(tx)

  #collect FUFI responce
  acc_new, gyro_new, theta_new, dt_new = get_measure(old_theta)
  return acc_new, gyro_new, theta_new, dt_new


# **Explanation** 🤯
**extract direction**

I take as ```motor_command``` the number of steps with sign to move the Cart of.

🤔 Should I convert it to motor angles??
 ```
 direction =  motor_command > 0
 ```
*    if ```motor_command``` is positive ➡  ```direction = True```, and we expect the Cart to go onward
*  if ```motor_command``` is negative ➡  ```direction = False```, and we expect the Cart to go backward.

**give Arduino instruction**
```
struct.pack("<?H", direction, int_motor_command)
```
The `struct.pack `function in Python is used to convert Python values into a bytes object according to a specified format string. Let's break down the format string `<?H` and the corresponding values `direction` and `int_motor_command`.


1.  Format String `<?H`:
  * `<` : Little-endian byte order.
  * `?` : Boolean value (1 byte).
  * `H` : Unsigned short (2 bytes).

2.  Values:
  * `direction` : A boolean value.
  * `int_motor_command` : An unsigned short integer.


Given this format string, you can use `struct.pack` to pack these values into a bytes object.

## Define what to do given the current state 🤔 🌠
At this point we should integrate the machine learning code to give him instructions. Let's try with something a little more dummy

In [ ]:
# dummy instruction
def dummy_instruction(theta):
  if theta > 0:
    return acc_new, gyro_new, theta_new, dt_new = step(-3, theta)
  else:
    return acc_new, gyro_new, theta_new, dt_new = step(3, theta)


## Actually tell him to move 🎆 🙏 🚑


In [ ]:
if __name__ == "__main__":

theta = 0
#current state
acc, gyro, theta, dt_new = get_measure(theta)

#loop function to move FUFI
while True:
  #decide what to do on the basis of current state
  acc, gyro, theta, dt_new = dummy_instruction(theta)